# Project 1: Recipe Assistant

In the Workflow Examples chapter, you built a **fixed recipe pipeline** — three LLM calls chained together in a strict order: simplify, adapt for kids, shopping list. You controlled every step.

Now let's build an **agent** that does all of that by deciding for itself. It will search for real recipes on the internet, present them, and generate a shopping list with cost estimates — choosing which tools to use and in what order.

| What you'll learn | Where it comes from |
|-------------------|--------------------|
| **HTTP Request Tool** — fetch live data from APIs | Builds on the Tool Calling chapter |
| **Sub-workflows as tools** — delegate tasks to specialist workflows | Builds on the First AI Agent chapter |
| **Agent orchestration** — the agent decides tool order | Builds on First AI Agent and Tool Calling |

**Workflows in this chapter:**

| File | What it does | GitHub Link |
|------|-------------|-------------|
| `10a_shopping_list_agent.json` | Sub-workflow: generates shopping list + cost estimate | [View](https://github.com/ezponda/ai-agents-course/blob/main/courses/n8n_no_code/book/_static/workflows/10a_shopping_list_agent.json) |
| `10b_recipe_assistant.json` | Orchestrator agent with HTTP search + sub-workflow tools | [View](https://github.com/ezponda/ai-agents-course/blob/main/courses/n8n_no_code/book/_static/workflows/10b_recipe_assistant.json) |

**Credentials needed:** OpenRouter API key (Settings → Credentials). TheMealDB API is free and needs no key.

---

## HTTP Request Tool

Until now, your agents used built-in tools (Calculator, Wikipedia). The **HTTP Request Tool** lets an agent call any web API — fetching live data from the internet.

| Property | Description |
|----------|-------------|
| **Purpose** | Let the agent make HTTP requests to any URL |
| **How it works** | You configure a URL template; the agent fills in parameters |
| **When to use** | Fetching data from external APIs (search, weather, prices, etc.) |
| **Credentials** | Depends on the API — some are free, others need API keys |

### TheMealDB: A Free Recipe API

We'll use [TheMealDB](https://www.themealdb.com/api.php) — a free recipe database that needs no API key.

| Endpoint | URL | What it returns |
|----------|-----|----------------|
| **Search by name** | `https://www.themealdb.com/api/json/v1/1/search.php?s={query}` | Recipes matching the search term |
| **Random recipe** | `https://www.themealdb.com/api/json/v1/1/random.php` | One random recipe |
| **Filter by category** | `https://www.themealdb.com/api/json/v1/1/filter.php?c={category}` | Recipes in a category (Chicken, Seafood, etc.) |

The search endpoint returns JSON with recipe details: name, ingredients (with quantities), instructions, and a thumbnail image.

**Docs:** [HTTP Request Tool](https://docs.n8n.io/integrations/builtin/cluster-nodes/sub-nodes/n8n-nodes-langchain.toolhttprequest/)

---

## Sub-Workflows as Tools

The **Call n8n Workflow Tool** (also called "Workflow Tool") lets an agent delegate work to another n8n workflow. Think of it as the agent hiring a specialist.

```
┌──────────────────────────────────────────────────────────────┐
│                    ORCHESTRATOR AGENT                        │
│                                                              │
│    "I found a recipe. Now I need a shopping list."           │
│    → Calls Shopping List Generator tool                      │
│                                                              │
└──────────────────────────┬───────────────────────────────────┘
                           │
                           ▼
┌──────────────────────────────────────────────────────────────┐
│                  SUB-WORKFLOW (separate workflow)             │
│                                                              │
│  Execute Workflow    Shopping & Cost     Output               │
│  Trigger         →  LLM             →                        │
│                                                              │
│  Receives recipe_text, returns shopping_list_with_cost       │
└──────────────────────────┬───────────────────────────────────┘
                           │
                           ▼
┌──────────────────────────────────────────────────────────────┐
│                    ORCHESTRATOR AGENT                        │
│                                                              │
│    ← Receives shopping list + cost estimate                  │
│    → Formats final response to user                          │
│                                                              │
└──────────────────────────────────────────────────────────────┘
```

| Property | Description |
|----------|-------------|
| **Purpose** | Delegate a task to a separate workflow |
| **Key detail** | The sub-workflow must start with **Execute Workflow Trigger** (not Manual Trigger) |
| **Input** | The agent sends data as fields defined in the tool configuration |
| **Output** | The sub-workflow's last node output is returned to the agent |

### Why Use Sub-Workflows?

| Benefit | Explanation |
|---------|-------------|
| **Separation of concerns** | Each workflow does one thing well |
| **Reusability** | The shopping list workflow can be called by other agents too |
| **Testability** | Test the sub-workflow independently before connecting it |
| **Readability** | The orchestrator stays simple; complexity is hidden in sub-workflows |

**Docs:** [Call n8n Workflow Tool](https://docs.n8n.io/integrations/builtin/cluster-nodes/sub-nodes/n8n-nodes-langchain.toolworkflow/)

---

## Workflow 1: Shopping List Agent (Sub-Workflow)

The "specialist" that the orchestrator delegates to. It receives a recipe and returns a shopping list grouped by store section with estimated costs.

```
┌────────────────────────┐     ┌────────────────────┐     ┌──────────────┐
│ Execute Workflow       │────▶│ Shopping & Cost     │────▶│ Output       │
│ Trigger                │     │ LLM                 │     │              │
└────────────────────────┘     └────────────────────┘     └──────────────┘
                                       ┊
                                  Chat Model
```

**File:** [`10a_shopping_list_agent.json`](https://github.com/ezponda/ai-agents-course/blob/main/courses/n8n_no_code/book/_static/workflows/10a_shopping_list_agent.json)

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/book/_static/workflows/10a_shopping_list_agent.json
> ```
>
> **Download:** {download}`10a_shopping_list_agent.json <_static/workflows/10a_shopping_list_agent.json>`

::::{dropdown} 🛠️ Build this workflow from scratch (step-by-step)
:color: secondary

### Step 1: Create a new workflow

1. Click **Workflows** → **Add Workflow**
2. Rename it to "Shopping List Agent"

### Step 2: Add the Execute Workflow Trigger

1. Delete the default Manual Trigger
2. Add **Execute Workflow Trigger** (search for "Execute Workflow Trigger")
3. In **Workflow Inputs**, add one field:

| Name | Type |
|------|------|
| `recipe_text` | `String` |

**Important:** This trigger is what allows the workflow to be called by another workflow. It defines the input the caller must provide.

### Step 3: Add the Basic LLM Chain

1. Add **Basic LLM Chain** → rename to `Shopping & Cost LLM`
2. Configure:
   - **Source for Prompt**: `Define below`
   - **Prompt** (Expression): `{{ $json.recipe_text }}`
   - **System Message**:
     ```
     You are a shopping assistant.
     From the recipe provided, create:
     1. A shopping list grouped by store section (Produce, Meat, Dairy, Pantry)
     2. Estimated price per item (typical US grocery prices)
     3. A grand total
     4. 2-3 budget tips

     Rules:
     - Include exact quantities from the recipe
     - Skip items most kitchens already have (salt, pepper, oil)
     - Output ONLY the shopping list and cost breakdown
     ```

### Step 4: Add the Chat Model (sub-node)

1. Click **+ Chat Model** at the bottom of the LLM Chain node
2. Select **OpenRouter Chat Model** (or OpenAI, etc.)
3. Choose your credential
4. Model: `openai/gpt-4o-mini`

### Step 5: Add Output

1. Add **Edit Fields** → rename to `Output`
2. Add field:
   - Name: `shopping_list_with_cost`
   - Value (Expression): `{{ $json.text }}`

### Step 6: Connect the nodes

```
Execute Workflow Trigger → Shopping & Cost LLM → Output
```

### Step 7: Test independently

1. Click **Test workflow**
2. Provide test data with a `recipe_text` field containing any recipe
3. Verify you get a grouped shopping list with prices

::::

### Node-by-Node Walkthrough

| Node | Type | What it does |
|------|------|-------------|
| **Execute Workflow Trigger** | Trigger | Receives `recipe_text` from the calling workflow |
| **Shopping & Cost LLM** | Basic LLM Chain | Generates shopping list + cost estimate from the recipe |
| **Output** | Set | Saves the result as `shopping_list_with_cost` |

**Sub-node:**

| Sub-node | Type | Purpose |
|----------|------|--------|
| **OpenRouter Chat Model** | Chat Model | Provides the LLM for generating the shopping list |

### System Message

```
You are a shopping assistant.
From the recipe provided, create:
1. A shopping list grouped by store section (Produce, Meat, Dairy, Pantry)
2. Estimated price per item (typical US grocery prices)
3. A grand total
4. 2-3 budget tips

Rules:
- Include exact quantities from the recipe
- Skip items most kitchens already have (salt, pepper, oil)
- Output ONLY the shopping list and cost breakdown
```

### Data Flow

```
INPUT                              OUTPUT
─────                              ──────
Execute Workflow Trigger: { recipe_text: "Chicken Parmesan — Ingredients: ..." }
    ↓
Shopping & Cost LLM: { text: "## Shopping List\n\n### Produce\n- 2 cups marinara sauce..." }
    ↓
Output: { shopping_list_with_cost: "## Shopping List\n\n### Produce\n...\n\nGrand Total: ~$18.50" }
```

::::{dropdown} 🔍 See detailed data transformation at each step
:color: info

```
┌──────────────────────────┐          ┌────────────────────┐          ┌──────────────────────┐
│ Execute Workflow Trigger │─────────▶│ Shopping & Cost LLM│─────────▶│ Output               │
└──────────────────────────┘          └────────────────────┘          └──────────────────────┘
          │                                    │                               │
          ▼                                    ▼                               ▼
┌──────────────────────┐           ┌──────────────────────┐        ┌───────────────────────────┐
│ {                    │           │ {                    │        │ {                         │
│   "recipe_text":     │           │   "text": "## Shop..│        │   "shopping_list_with_cost"│
│   "Chicken Parmesan  │           │   ping List\n..."   │        │   : "## Shopping List\n..."│
│    — Ingredients..." │           │ }                    │        │ }                         │
│ }                    │           │                      │        │                           │
└──────────────────────┘           └──────────────────────┘        └───────────────────────────┘
```

**Key insight:** The Basic LLM Chain outputs to `text`. The Output node renames it to `shopping_list_with_cost` so the calling workflow gets a clearly named field.

::::

---

## Workflow 2: Recipe Assistant (Orchestrator)

The main agent with two tools: an HTTP Request for recipe search and a Call n8n Workflow for the shopping list.

```
┌──────────────┐     ┌──────────────┐     ┌──────────────────────┐     ┌──────────────┐
│ Manual       │────▶│ Input —      │────▶│ Recipe Assistant     │────▶│ Output       │
│ Trigger      │     │ Request      │     │ Agent                │     │              │
└──────────────┘     └──────────────┘     └──────────────────────┘     └──────────────┘
                                                    ┊ sub-nodes
                                          ┌─────────┼─────────┐
                                          ┊         ┊         ┊
                                     Chat Model  HTTP Req   Call Workflow
                                                 (TheMealDB) (Shopping List)
```

**File:** [`10b_recipe_assistant.json`](https://github.com/ezponda/ai-agents-course/blob/main/courses/n8n_no_code/book/_static/workflows/10b_recipe_assistant.json)

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/book/_static/workflows/10b_recipe_assistant.json
> ```
>
> **Download:** {download}`10b_recipe_assistant.json <_static/workflows/10b_recipe_assistant.json>`

```{note}
After importing, you need to configure the **Shopping List Generator** tool to point to your imported Shopping List Agent workflow. Open the tool node and select the workflow from the dropdown.
```

**Setup checklist** — verify before testing:

- [ ] Both workflows imported (`10a` Shopping List Agent + `10b` Recipe Assistant)
- [ ] **Shopping List Generator** tool node points to your Shopping List Agent workflow (open the tool → select it from the dropdown)
- [ ] Shopping List Agent starts with **Execute Workflow Trigger** (not Manual Trigger)
- [ ] OpenRouter credential configured on both workflows

::::{dropdown} 🛠️ Build this workflow from scratch (step-by-step)
:color: secondary

### Step 1: Create a new workflow

1. Click **Workflows** → **Add Workflow**
2. Rename it to "Recipe Assistant"

### Step 2: Add the trigger and input

1. Add **Manual Trigger**
2. Add **Edit Fields** → rename to `Input — Request`
3. Add one String field:

| Name | Value |
|------|-------|
| `chatInput` | `I want to cook something with chicken` |

### Step 3: Add the AI Agent node

1. Add **AI Agent** → rename to `Recipe Assistant Agent`
2. Configure:
   - **Source for Prompt**: `Define below`
   - **Prompt** (Expression): `{{ $json.chatInput }}`
   - **System Message** (in Options):
     ```
     You are a friendly recipe assistant.

     Process:
     1. Use Recipe Search to find a recipe matching the user's request
     2. Present the recipe clearly (name, ingredients, instructions)
     3. Use Shopping List Generator to create a shopping list with cost estimate

     Rules:
     - Always search first — never invent recipes
     - After finding a recipe, always generate a shopping list + cost
     - If no results, suggest alternative search terms
     - Keep your response friendly and organized
     ```

### Step 4: Add the Chat Model (sub-node)

1. Click **+ Chat Model** at the bottom of the AI Agent node
2. Select **OpenRouter Chat Model** (or OpenAI, etc.)
3. Choose your credential
4. Model: `openai/gpt-4o-mini`

### Step 5: Add the HTTP Request Tool (sub-node)

1. Click **+ Tool** at the bottom of the AI Agent node
2. Select **HTTP Request Tool**
3. Rename to `Recipe Search`
4. Configure:

| Setting | Value |
|---------|-------|
| **Name** | `Recipe Search` |
| **Description** | `Search for recipes by name or ingredient. Input should be a search term like 'chicken' or 'pasta'. Returns recipe details including name, ingredients, and instructions.` |
| **Method** | `GET` |
| **URL** | `https://www.themealdb.com/api/json/v1/1/search.php` |
| **Send Query Parameters** | Enabled |
| **Specify Query Parameters** | `Using Fields Below` |

**Query Parameters:**

| Name | Value |
|------|-------|
| `s` | `{query}` |

**Placeholder Definitions:**

| Name | Description |
|------|-------------|
| `query` | `The recipe search term (e.g. 'chicken', 'pasta', 'chocolate cake')` |

5. Enable **Optimize Response** to reduce token usage

### Step 6: Add the Call n8n Workflow Tool (sub-node)

1. Click **+ Tool** at the bottom of the AI Agent node
2. Select **Call n8n Workflow Tool**
3. Rename to `Shopping List Generator`
4. Configure:

| Setting | Value |
|---------|-------|
| **Name** | `Shopping List Generator` |
| **Description** | `Generate a shopping list with cost estimates from a recipe. Input should be the full recipe text including ingredients and instructions.` |
| **Workflow** | Select your imported "Shopping List Agent" workflow |

**Workflow Fields:**

| Name | Value |
|------|-------|
| `recipe_text` | `{recipe_text}` |

### Step 7: Add Output

1. Add **Edit Fields** → rename to `Output`
2. Add field:
   - Name: `answer`
   - Value (Expression): `{{ $json.output }}`

### Step 8: Connect the main nodes

```
Manual Trigger → Input — Request → Recipe Assistant Agent → Output
```

### Step 9: Test

1. Click **Execute Workflow**
2. Watch the agent: it should call Recipe Search first, then Shopping List Generator
3. Expand execution details to see the tool call sequence

::::

### Node-by-Node Walkthrough

<div style="overflow: auto; max-height: 200px; border: 1px solid #ddd; border-radius: 4px; padding: 10px; margin-bottom: 15px; background: #f8f8f8;">
<pre style="margin: 0; white-space: pre;">
┌──────────────────┐     ┌──────────────────┐     ┌────────────────────────┐     ┌──────────────┐
│  Run: Recipe     │────▶│  Input — Request │────▶│  Recipe Assistant      │────▶│  Output      │
│  Assistant       │     │                  │     │  Agent                 │     │              │
└──────────────────┘     └──────────────────┘     └────────────────────────┘     └──────────────┘
                                                             ┊
                                                             ┊ (sub-nodes)
                                              ┌──────────────┼──────────────┐
                                              ┊              ┊              ┊
                                       ┌─────────────┐ ┌──────────┐ ┌───────────────────┐
                                       │ Chat Model  │ │ Recipe   │ │ Shopping List     │
                                       │             │ │ Search   │ │ Generator         │
                                       └─────────────┘ └──────────┘ └───────────────────┘
</pre>
</div>

| Node | Type | What it does |
|------|------|-------------|
| **Run: Recipe Assistant** | Manual Trigger | Starts the workflow |
| **Input — Request** | Set | Creates `chatInput` field with the user's request |
| **Recipe Assistant Agent** | AI Agent | Receives `{{ $json.chatInput }}`, decides which tools to call, outputs `output` |
| **Output** | Set | Saves `{{ $json.output }}` as `answer` |

**Sub-nodes (dotted lines to the Agent):**

| Sub-node | Type | Purpose |
|----------|------|--------|
| **OpenRouter Chat Model** | Chat Model | Provides the AI "brain" |
| **Recipe Search** | HTTP Request Tool | Searches TheMealDB API for recipes |
| **Shopping List Generator** | Call n8n Workflow Tool | Invokes the Shopping List Agent sub-workflow |

### System Message

```
You are a friendly recipe assistant.

Process:
1. Use Recipe Search to find a recipe matching the user's request
2. Present the recipe clearly (name, ingredients, instructions)
3. Use Shopping List Generator to create a shopping list with cost estimate

Rules:
- Always search first — never invent recipes
- After finding a recipe, always generate a shopping list + cost
- If no results, suggest alternative search terms
- Keep your response friendly and organized
```

### Tool Descriptions

Tool descriptions tell the agent **when** and **how** to use each tool:

| Tool | Description given to the agent |
|------|-------------------------------|
| **Recipe Search** | `Search for recipes by name or ingredient. Input should be a search term like 'chicken' or 'pasta'. Returns recipe details including name, ingredients, and instructions.` |
| **Shopping List Generator** | `Generate a shopping list with cost estimates from a recipe. Input should be the full recipe text including ingredients and instructions.` |

These descriptions are critical — they're how the agent knows what each tool does and what input to provide. Vague descriptions lead to wrong tool choices.

### Data Flow

```
INPUT                          AGENT ACTIONS                        OUTPUT
─────                          ─────────────                        ──────
Trigger: { }
    ↓
Input: { chatInput: "I want to cook something with chicken" }
    ↓
Agent:
  1. Calls Recipe Search (HTTP) → gets Chicken Parmesan from TheMealDB
  2. Calls Shopping List Generator (sub-workflow) → gets list + costs
  3. Formats final response: recipe + shopping list + cost estimate
    ↓
Output: { answer: "Here's a great recipe! ...\n\nShopping List:\n...\n\nEstimated Total: ~$18.50" }
```

::::{dropdown} 🔍 See detailed data transformation at each step
:color: info

```
┌──────────────────┐     ┌──────────────────┐     ┌────────────────────────┐     ┌──────────────┐
│  Manual Trigger  │────▶│  Input — Request │────▶│  Recipe Assistant      │────▶│  Output      │
└──────────────────┘     └──────────────────┘     │  Agent                 │     └──────────────┘
                                                  └────────────────────────┘
         │                       │                          │                          │
         ▼                       ▼                          ▼                          ▼
┌────────────────┐   ┌───────────────────────┐   ┌──────────────────────┐   ┌──────────────────┐
│ { }            │   │ {                     │   │ TOOL CALL 1:         │   │ {                │
│                │   │   "chatInput": "I want│   │ Recipe Search        │   │   "answer": "... │
│                │   │   to cook something   │   │ → TheMealDB API      │   │   recipe +       │
│                │   │   with chicken"       │   │ → Chicken Parmesan   │   │   shopping list + │
│                │   │ }                     │   │                      │   │   cost estimate"  │
│                │   │                       │   │ TOOL CALL 2:         │   │ }                │
│                │   │                       │   │ Shopping List Gen.   │   │                  │
│                │   │                       │   │ → Sub-workflow       │   │                  │
│                │   │                       │   │ → Grouped list +     │   │                  │
│                │   │                       │   │   cost estimate      │   │                  │
│                │   │                       │   │                      │   │                  │
│                │   │                       │   │ { "output": "Here's  │   │                  │
│                │   │                       │   │   a great recipe.." }│   │                  │
└────────────────┘   └───────────────────────┘   └──────────────────────┘   └──────────────────┘
```

**Key insight:** The agent made two tool calls in sequence. It searched for a recipe first (HTTP Request), then delegated the shopping list creation to the sub-workflow. The agent decided this order by itself — you didn't hard-code it.

::::

### What to Observe

After running the workflow:

1. Click **Recipe Assistant Agent** → expand the execution details
2. You should see **two tool calls** in order:
   - First: **Recipe Search** → the agent searched TheMealDB
   - Second: **Shopping List Generator** → the agent delegated to the sub-workflow
3. The final output combines: recipe name + ingredients + instructions + shopping list + cost estimate
4. Notice: you didn't code "step 1, step 2" — the agent figured out the right order from the system message

---

## Full Circle: From Fixed Chain to Autonomous Agent

Compare what you built in the Prompt Chaining section with what you have now:

| | Prompt Chaining Pattern | This Chapter: Recipe Agent |
|--|---------------------------|---------------------------|
| **Recipe source** | Hard-coded in the Input node | Fetched live from the internet |
| **Step order** | Fixed: Step 1 → Step 2 → Step 3 | Agent decides the order |
| **Shopping list** | One of three chained steps | Delegated to a specialist sub-workflow |
| **Cost estimate** | Not included | Included automatically |
| **Flexibility** | Only works for the recipe you paste in | Works for any recipe the user asks about |
| **Nodes** | 8+ nodes in a rigid chain | 2 workflows, agent orchestrates |

The fixed chain is reliable and predictable. The agent is flexible and autonomous. Both have their place — the right choice depends on your use case.

---

## Try It Yourself

These challenges extend the Recipe Assistant. Each one practices a concept from earlier in the course.

### Challenge 1: Chat Version

Swap the Manual Trigger for a **Chat Trigger** and add **Memory** to the agent. This lets you have a conversation:

- "I want chicken" → agent finds recipe + shopping list
- "What about a vegetarian option?" → agent remembers context, searches again
- "Compare the costs" → agent uses memory to compare both shopping lists

**Hint:** Follow the same pattern from Example 3 in the First AI Agent chapter (Chat Trigger + Memory). Remember the output field must be named `output`.

**Done when:** you can ask a follow-up question and the agent references your previous request without you repeating it.

### Challenge 2: Random Recipe Tool

Add a third tool using TheMealDB's random endpoint:

```
https://www.themealdb.com/api/json/v1/1/random.php
```

This takes no parameters — it returns a random recipe. Update the system message so the agent uses this tool when the user says "surprise me" or "pick something random."

**Done when:** saying "surprise me" triggers the random endpoint and returns a complete answer (recipe + shopping list).

### Challenge 3: Category Filter Tool

Add a tool using TheMealDB's category filter:

```
https://www.themealdb.com/api/json/v1/1/filter.php?c={category}
```

Categories include: `Chicken`, `Seafood`, `Vegetarian`, `Dessert`, `Beef`, `Pasta`. The agent can use this to narrow down options before searching for a specific recipe.

**Done when:** asking "show me seafood options" returns a list of recipes from the Seafood category.

### Challenge 4: Kid-Friendly Adaptation

Add a third tool (or expand the Shopping List sub-workflow) that adapts recipes for kids — milder flavors, fun step names, safety notes. This connects back to the Prompt Chaining pattern in the Workflow Examples chapter.

**Done when:** asking "make it kid-friendly" after a recipe returns simplified instructions with safety notes.

---

## Summary

| Concept | What You Learned |
|---------|------------------|
| **HTTP Request Tool** | Agents can fetch live data from any web API |
| **Call n8n Workflow Tool** | Agents can delegate tasks to specialist sub-workflows |
| **Execute Workflow Trigger** | Sub-workflows use this trigger (not Manual Trigger) to receive data |
| **Tool descriptions** | Clear descriptions tell the agent when and how to use each tool |
| **Agent orchestration** | The agent decides tool order from the system message — no hard-coded steps |

**Key expressions:**
- `{{ $json.chatInput }}` — pass the user's request to the agent
- `{{ $json.output }}` — access the agent's response
- `{{ $json.text }}` — access a Basic LLM Chain's response
- `{{ $json.recipe_text }}` — access the input in the sub-workflow

**Docs:**
- [HTTP Request Tool](https://docs.n8n.io/integrations/builtin/cluster-nodes/sub-nodes/n8n-nodes-langchain.toolhttprequest/)
- [Call n8n Workflow Tool](https://docs.n8n.io/integrations/builtin/cluster-nodes/sub-nodes/n8n-nodes-langchain.toolworkflow/)
- [Execute Workflow Trigger](https://docs.n8n.io/integrations/builtin/core-nodes/n8n-nodes-base.executeworkflowtrigger/)
- [TheMealDB API](https://www.themealdb.com/api.php)